In [1]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model


MODEL_PATH = 'models/eye_status_model.h5'
model = load_model(MODEL_PATH)
IMG_SIZE = 24
CLASS_LABELS = ['Mata Tertutup', 'Mata Terbuka']

# Inisialisasi MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1, # Deteksi satu wajah saja untuk efisiensi
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Landmark mata (berdasarkan dokumentasi MediaPipe)
# https://github.com/google/mediapipe/blob/master/mediapipe/modules/face_geometry/data/canonical_face_model_uv_visualization.png
LEFT_EYE_IDXS = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE_IDXS = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

# Fungsi untuk memproses dan memprediksi
def preprocess_and_predict(eye_img):
    # Pre-proccess
    gray_eye = cv2.cvtColor(eye_img, cv2.COLOR_BGR2GRAY)
    resized_eye = cv2.resize(gray_eye, (IMG_SIZE, IMG_SIZE))
    normalized_eye = resized_eye / 255.0
    input_eye = np.expand_dims(np.expand_dims(normalized_eye, axis=-1), axis=0)
    
    # Prediksi
    prediction = model.predict(input_eye)
    return prediction[0][0]

# LOOP WEBCAM

cap = cv2.VideoCapture(0)
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
        
    # Balik gambar & konversi BGR ke RGB untuk MediaPipe
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Proses frame dengan MediaPipe
    results = face_mesh.process(rgb_frame)

    # Jika landmark terdeteksi
    if results.multi_face_landmarks:
        # Ambil landmark dari wajah pertama yang terdeteksi
        face_landmarks = results.multi_face_landmarks[0].landmark
        
        # Proses mata kiri
        left_eye_points = np.array([[face_landmarks[i].x * frame.shape[1], face_landmarks[i].y * frame.shape[0]] for i in LEFT_EYE_IDXS]).astype(int)
        lx_min, ly_min = np.min(left_eye_points, axis=0)
        lx_max, ly_max = np.max(left_eye_points, axis=0)
        
        # Proses mata kanan
        right_eye_points = np.array([[face_landmarks[i].x * frame.shape[1], face_landmarks[i].y * frame.shape[0]] for i in RIGHT_EYE_IDXS]).astype(int)
        rx_min, ry_min = np.min(right_eye_points, axis=0)
        rx_max, ry_max = np.max(right_eye_points, axis=0)
        
        # Tambahkan sedikit padding
        padding = 5
        
        # Potong gambar mata kiri
        if ly_max - ly_min > 0 and lx_max - lx_min > 0:
            left_eye_crop = frame[ly_min-padding:ly_max+padding, lx_min-padding:lx_max+padding]
            if left_eye_crop.size > 0:
                prob_left = preprocess_and_predict(left_eye_crop)
                status_left = CLASS_LABELS[1] if prob_left > 0.5 else CLASS_LABELS[0]
                color_left = (0, 255, 0) if status_left == 'Mata Terbuka' else (0, 0, 255)
                cv2.rectangle(frame, (lx_min-padding, ly_min-padding), (lx_max+padding, ly_max+padding), color_left, 2)
                cv2.putText(frame, status_left, (lx_min-padding, ly_min-15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_left, 2)

        # Potong gambar mata kanan
        if ry_max - ry_min > 0 and rx_max - rx_min > 0:
            right_eye_crop = frame[ry_min-padding:ry_max+padding, rx_min-padding:rx_max+padding]
            if right_eye_crop.size > 0:
                prob_right = preprocess_and_predict(right_eye_crop)
                status_right = CLASS_LABELS[1] if prob_right > 0.5 else CLASS_LABELS[0]
                color_right = (0, 255, 0) if status_right == 'Mata Terbuka' else (0, 0, 255)
                cv2.rectangle(frame, (rx_min-padding, ry_min-padding), (rx_max+padding, ry_max+padding), color_right, 2)
                cv2.putText(frame, status_right, (rx_min-padding, ry_min-15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_right, 2)

    cv2.imshow('Deteksi Kantuk dengan MediaPipe', frame)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━